In [1]:
import pandas as pd
import numpy as np
import os
import pyarrow as pa
import pyarrow.parquet as pq

### 因子构造样例

#### 预先读取merge_daily_info

In [2]:
merge_daily_info = pd.read_parquet(r'/Users/xuyanye/Desktop/quant_mm/factor/data/merge_daily_info.parquet')
merge_daily_info['TradingDate'] = pd.to_datetime(merge_daily_info['TradingDate'])
merge_daily_info = merge_daily_info.drop_duplicates(subset=['Stkcd','TradingDate'])
TRD_Dalyr = pd.read_parquet(r'/Users/xuyanye/Desktop/quant_mm/factor/data/TRD_Dalyr.parquet')

In [3]:
TRD_Dalyr.columns

Index(['Stkcd', 'TradingDate', 'Opnprc', 'Hiprc', 'Loprc', 'Clsprc',
       'Dnshrtrd', 'Dnvaltrd', 'Dsmvosd', 'Dsmvtll', 'Dretwd', 'Dretnd',
       'Adjprcwd', 'Adjprcnd', 'Markettype', 'Capchgdt', 'Trdsta',
       'Ahshrtrd_D', 'Ahvaltrd_D', 'PreClosePrice', 'ChangeRatio', 'LimitDown',
       'LimitUp', 'LimitStatus'],
      dtype='object')

In [14]:
#数据提取与合并
Hiprc = TRD_Dalyr[['TradingDate','Stkcd','Hiprc']]
Loprc = TRD_Dalyr[['TradingDate','Stkcd','Loprc']]
Clsprc = TRD_Dalyr[['TradingDate','Stkcd','Clsprc']]
calc_cvturn = pd.merge(Hiprc,Loprc ,on=['TradingDate','Stkcd'],how='left')
calc_cvturn = pd.merge(calc_cvturn,Clsprc ,on=['TradingDate','Stkcd'],how='left')
calc_cvturn.columns

Index(['TradingDate', 'Stkcd', 'Hiprc', 'Loprc', 'Clsprc'], dtype='object')

# alpha053

In [15]:
COLUMN_FACTOR_NAME = "alpha053" 
ALPHA_SAVE_PATH_DIR = "/Users/xuyanye/Desktop/quant mm/factor/alpha"
if not os.path.exists(ALPHA_SAVE_PATH_DIR):
    os.makedirs(ALPHA_SAVE_PATH_DIR)
# 定义最终因子在DataFrame中的列名，以及Parquet文件的基础名称
# 这两部分必须一致，才能被您的回测代码正确识别

In [16]:
# 假设数据已加载到 calc_factor_data 中
calc_factor_data = calc_cvturn.copy()

# 1. 确保数据按股票代码和交易日期排序
calc_factor_data = calc_factor_data.sort_values(by=['Stkcd', 'TradingDate']).copy()

# 2. 计算 (close - low)
calc_factor_data['close_minus_low'] = calc_factor_data['Clsprc'] - calc_factor_data['Loprc']

# 3. 计算 (high - close)
calc_factor_data['high_minus_close'] = calc_factor_data['Hiprc'] - calc_factor_data['Clsprc']

# 4. 计算 (close - low) / (high - close)
calc_factor_data['close_low_ratio'] = (calc_factor_data['close_minus_low']) / (calc_factor_data['high_minus_close'])

# 5. 计算 delta(((close-low) - (high-close)) / (close-low), 9)
calc_factor_data['delta_factor'] = calc_factor_data.groupby('Stkcd')['close_low_ratio'].transform(lambda x: x.diff(9))

# 6. 乘以 -1
calc_factor_data['final_factor'] = -1 * calc_factor_data['delta_factor']

# 7. 处理可能的 NaN 或 inf 值
calc_factor_data['final_factor'].replace([float('inf'), float('-inf')], 0, inplace=True)


/var/folders/6j/sy28sk9165s6btqff5dbsdmc0000gn/T/ipykernel_91506/497066727.py:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  calc_factor_data['final_factor'].replace([float('inf'), float('-inf')], 0, inplace=True)


In [17]:
# 假设 merge_daily_info 是另一个包含 Stkcd 和 TradingDate 的 DataFrame
# 将计算出的 Alpha08_Factor 合并到 merge_daily_info
# 注意：如果 calc_cvturn 就是最终要用的 DataFrame，可能不需要这一步合并
# 这里为了与用户提供的格式保持一致，假设有一个目标 DataFrame 'merge_daily_info'
# 如果 calc_cvturn 本身就是最终结果，则可以跳过此合并步骤
# 请根据实际情况调整 'merge_daily_info' 的定义
try:
    # 模拟 merge_daily_info 如果它不存在
    if 'merge_daily_info' not in locals():
        print("警告：'merge_daily_info' 未定义。为演示目的，将假设 merge_daily_info 等同于 calc_cvturn 的基础部分。")
        merge_daily_info = calc_cvturn[['Stkcd', 'TradingDate']].copy() # 只是一个示例

    # 进行合并
    calc_cvturn = pd.merge(merge_daily_info,
                           calc_factor_data[['Stkcd','TradingDate','final_factor']],
                           on=['Stkcd','TradingDate'],
                           how='left')
    calc_cvturn = calc_cvturn.rename(columns={'final_factor': 'alpha053'})
except NameError:
    print("错误：'merge_daily_info' 未定义，请确保该DataFrame存在后再执行合并。")


# 去掉缺失值与inf (根据用户提供的格式)
# 这一步会移除 Alpha12_Factor 列中因为 delta 计算导致的 NaN 值 (例如每个股票的第一个交易日)
calc_cvturn = calc_cvturn.dropna(subset=['alpha053']) # 只针对因子列去除NaN，避免影响其他数据
calc_cvturn = calc_cvturn[~np.isinf(calc_cvturn['alpha053'])] # 确保因子值不是无穷大

# 打印结果 DataFrame 的头部，以供检查
print(calc_cvturn.head())

    Stkcd TradingDate  alpha053
94      1  1991-08-28  0.000000
95      1  1991-08-29  0.000000
96      1  1991-08-30 -0.909091
97      1  1991-08-31 -1.750000
98      1  1991-09-02  0.000000


In [19]:
##采用parquet优化内存
save_path = r'/Users/xuyanye/Desktop/quant_mm/factor/alpha/alpha053.parquet'
calc_cvturn.to_parquet(save_path, compression='gzip')